# Face Generation with GANs

* [Step 1](#step1): Reading the CelebA dataset
* [Step 2](#step2): Building inputs to the networks
* [Step 3](#step3): Building the Generator Network
* [Step 4](#step4): Building the Discriminator Network
* [Step 5](#step5): Putting all together
* [Step 6](#step6): Training
* [Step 7](#step7): Viewing Results

<a id='step1'></a>
## Step 1: Reading the CelebA dataset